In [1]:
import functools
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from tensorflow import keras
import tensorflow_text as text


In [2]:
TRAIN_DATA_URL = "http://localhost:8890/lab/tree/Desktop/Code/final_project_twitter/server/tweet3.csv"
TEST_DATA_URL = "http://localhost:8890/lab/tree/Desktop/Code/final_project_twitter/server/test_data.csv"


train_file_path = tf.keras.utils.get_file("tweets3.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("test_tweets.csv", TRAIN_DATA_URL)

In [3]:
np.set_printoptions(precision=3, suppress=True)

In [4]:
!head {train_file_path}

mongo_id,ratio,date,username,to,replies,retweets,favorites,text,geo,mentions,hashtags,id,permalink,modified,percentage,sentiment
{'$oid': '5e84b6da3b636ca8068a302b'},0.041947926711668276,{'$date': '2019-11-13T03:56:26.000+0000'},BernieSanders,,87,565,2074,"""Today we say thank you to our grassroots donors who make this movement possible. No billionaires", super PACs or corporate lobbyists are supporting us," and we wouldn’t have it any other way.https://www.pscp.tv/w/cJswGjMyNzU3OTl8MXlwS2RCcmdsVlJKV84SNzDXOW11HYezwTRzB7dVUPXV6YNFz9Jc4eayeHOZ …""",,,,{'$date': '2020-04-01T20:32:33.307+0000'},4.1947926711668275,2.0
{'$oid': '5e84b6da3b636ca8068a302c'},0.018886621976100262,2019-11-12 22:31:09,BernieSanders,RVAwonk,324.0,3112.0,17155.0,"""Stephen Miller's white nationalist views are a danger to the American people. We are going to defeat this hateful administration and everything it stands for.https://twitter.com/RVAwonk/status/1194316047383433227 …""",,,,1194382146850476040,https://twitt

In [5]:
!head {test_file_path}

mongo_id,ratio,date,username,to,replies,retweets,favorites,text,geo,mentions,hashtags,id,permalink,modified,percentage,sentiment
{'$oid': '5e84b6da3b636ca8068a302b'},0.041947926711668276,{'$date': '2019-11-13T03:56:26.000+0000'},BernieSanders,,87,565,2074,"""Today we say thank you to our grassroots donors who make this movement possible. No billionaires", super PACs or corporate lobbyists are supporting us," and we wouldn’t have it any other way.https://www.pscp.tv/w/cJswGjMyNzU3OTl8MXlwS2RCcmdsVlJKV84SNzDXOW11HYezwTRzB7dVUPXV6YNFz9Jc4eayeHOZ …""",,,,{'$date': '2020-04-01T20:32:33.307+0000'},4.1947926711668275,2.0
{'$oid': '5e84b6da3b636ca8068a302c'},0.018886621976100262,2019-11-12 22:31:09,BernieSanders,RVAwonk,324.0,3112.0,17155.0,"""Stephen Miller's white nationalist views are a danger to the American people. We are going to defeat this hateful administration and everything it stands for.https://twitter.com/RVAwonk/status/1194316047383433227 …""",,,,1194382146850476040,https://twitt

In [6]:
LABEL_COLUMN = 'ratio'
# LABELS = [0.0]


In [7]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
    # column_names=['_id','ratio','date','username','to','replies','retweets','favorites','text','geo','mentions','hashtags','id', 'permalink','modified','percentage','sentiment'],
#     column_defaults= [tf.string, tf.float32, tf.string, tf.string, tf.string, tf.float32, tf.int32, tf.float32, tf.float32, tf.string, tf.string, tf.string, tf.string, tf.string, tf.string, tf.string, tf.float32 ],
      na_value="?",
      num_epochs=1,
      ignore_errors=True,
      **kwargs
      )
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


In [8]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [9]:
show_batch(raw_train_data)


mongo_id            : [b"{'$oid': '5e84b7523b636ca8068a4728'}"
 b"{'$oid': '5e84b7523b636ca8068a476c'}"
 b"{'$oid': '5e84b7523b636ca8068a40bb'}"
 b"{'$oid': '5e84b6da3b636ca8068a3522'}"
 b"{'$oid': '5e84b7523b636ca8068a45c8'}"]
date                : [b'2019-03-28 11:04:04' b'2019-03-19 12:24:21' b'2019-08-12 21:26:32'
 b'2019-07-18 20:00:16' b'2019-05-02 16:29:21']
username            : [b'realDonaldTrump' b'realDonaldTrump' b'realDonaldTrump' b'BernieSanders'
 b'realDonaldTrump']
to                  : [b'' b'' b'' b'' b'']
replies             : [26110. 33157. 12926.  3159.  9825.]
retweets            : [28192. 31972. 18101.  8134. 12782.]
favorites           : [127208. 126349.  75822.  54828.  69578.]
text                : [b'"Wow'
 b'"The Fake News Media has NEVER been more Dishonest or Corrupt than it is right now. There has never been a time like this in American History. Very exciting but also'
 b'"The United States is learning much from the failed missile explosion in Russia. We 

In [10]:
temp_dataset = get_dataset(train_file_path)

show_batch(temp_dataset)

mongo_id            : [b"{'$oid': '5e84b6da3b636ca8068a3195'}"
 b"{'$oid': '5e84b7523b636ca8068a3b97'}"
 b"{'$oid': '5e84b6da3b636ca8068a3285'}"
 b"{'$oid': '5e84b6da3b636ca8068a3352'}"
 b"{'$oid': '5e84b7523b636ca8068a45a0'}"]
date                : [b'2019-10-12 22:46:14' b'2019-10-28 20:02:56' b'2019-09-17 15:30:00'
 b'2019-08-31 16:36:00' b'2019-05-06 03:05:35']
username            : [b'BernieSanders' b'realDonaldTrump' b'BernieSanders' b'BernieSanders'
 b'realDonaldTrump']
to                  : [b'' b'' b'' b'' b'']
replies             : [  453. 54043.   334.   414. 22833.]
retweets            : [  1726. 131069.   1324.   1787.  24943.]
favorites           : [  6678. 574129.   7786.   6758. 127311.]
text                : [b'"I\'m excited to announce that we will be holding a major rally in New York next Saturday'
 b'"We have declassified a picture of the wonderful dog (name not declassified) that did such a GREAT JOB in capturing and killing the Leader of ISIS'
 b'"When one person 

In [11]:
SELECT_COLUMNS = ['ratio','username', 'replies', 'retweets','favorites', 'text', 'percentage']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

username            : [b'realDonaldTrump' b'realDonaldTrump' b'realDonaldTrump'
 b'realDonaldTrump' b'BernieSanders']
replies             : [14420.  9067. 10548.  3936.    80.]
retweets            : [21465. 25686. 16275. 10031.   715.]
favorites           : [ 92834. 116703.  82471.  49162.   2337.]
text                : [b'"....all deliveries of Fentanyl from China (or anywhere else!). Fentanyl kills 100'
 b'"Just wrapped up a great meeting with my friend Prime Minister @NarendraModi of India at the #G7Summit in Biarritz'
 b'"S&P closes at Record High!https://twitter.com/realDonaldTrump/status/1141706991464849408 \xe2\x80\xa6"'
 b'"....pass the laugh test'
 b'"Haim Saban is used to buying elections for his candidates. He and other billionaires oppose our campaign because they know they can\'t buy us. We don\'t want their money.pic.twitter.com/IMV2RwJOxj"']
percentage          : [15.533  7.769 12.79   8.006  3.423]


In [12]:
SELECT_COLUMNS = [ 'ratio','replies', 'favorites', 'retweets', 'percentage' ]
DEFAULTS = [0.0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS, column_defaults = DEFAULTS)

show_batch(temp_dataset)


replies             : [2895. 7194.  279. 2851.  130.]
retweets            : [ 9527. 25373.  1021.  8238.   833.]
favorites           : [48718. 76743.  4989. 41771.  3097.]
percentage          : [5.942 9.374 5.592 6.825 4.198]


In [13]:
example_batch, labels_batch = next(iter(temp_dataset)) 

In [14]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

In [15]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[13143.    19430.    80076.       16.413]
 [  515.     1441.     9671.        5.325]
 [ 7980.    17342.    64201.       12.43 ]
 [ 5238.    13041.    47685.       10.985]
 [  219.      891.     5294.        4.137]]

[0.164 0.053 0.124 0.11  0.041]


In [16]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [17]:
NUMERIC_FEATURES = ['replies', 'favorites', 'retweets', 'percentage']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [18]:
show_batch(packed_train_data)

mongo_id            : [b"{'$oid': '5e84b6da3b636ca8068a36d2'}"
 b"{'$oid': '5e84b7523b636ca8068a3fd4'}"
 b"{'$oid': '5e84b6da3b636ca8068a3243'}"
 b"{'$oid': '5e84b7523b636ca8068a46f2'}"
 b"{'$oid': '5e84b7523b636ca8068a4611'}"]
date                : [b'2019-06-09 20:21:45' b'2019-08-27 23:36:43' b'2019-09-22 15:05:00'
 b'2019-04-03 17:09:19' b'2019-04-24 13:47:54']
username            : [b'BernieSanders' b'realDonaldTrump' b'BernieSanders' b'realDonaldTrump'
 b'realDonaldTrump']
to                  : [b'' b'' b'' b'' b'']
text                : [b'"Some candidates believe the best way forward is a ""middle ground"" strategy that stands up to nobody and changes nothing. That approach isn\'t just bad policy'
 b'"Can you believe it? I\xe2\x80\x99m at 94% approval in the Republican Party'
 b'"McDonald\'s made $6 billion in profits last year and paid its CEO $22 million. It damn well can afford to pay its workers a living wage. We will take on greedy corporations and raise wages for workers 

In [19]:
example_batch, labels_batch = next(iter(packed_train_data)) 

In [20]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe(include = 'all')
desc

,replies,favorites,retweets,percentage
count,6610.000000,6.610000e+03,6610.000000,6610.000000
mean,9573.276097,5.765821e+04,12982.077912,12.709540
std,12778.795593,5.423316e+04,12568.021772,9.656935
min,2.000000,1.220000e+02,42.000000,0.667503
25%,530.000000,9.192250e+03,1946.500000,5.923261
50%,5187.500000,5.444050e+04,12524.500000,10.175822
75%,13543.000000,8.659375e+04,19620.000000,16.928996
max,129067.000000,1.021327e+06,245074.000000,200.801833


In [21]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [22]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

In [23]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x633751e60>, mean=array([ 9573.276, 57658.211, 12982.078,    12.71 ]), std=array([12778.796, 54233.158, 12568.022,     9.657])))

In [24]:
example_batch['numeric']


<tf.Tensor: id=660, shape=(5, 4), dtype=float32, numpy=
array([[  6489.   ,  70129.   ,  12986.   ,      9.253],
       [ 26057.   , 105041.   ,  17053.   ,     24.807],
       [ 15522.   , 138200.   ,  23952.   ,     11.232],
       [   227.   ,   4623.   ,   1083.   ,      4.91 ],
       [  9721.   ,  71799.   ,  14529.   ,     13.539]], dtype=float32)>

In [25]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.241,  0.23 ,  0.   , -0.358],
       [ 1.29 ,  0.874,  0.324,  1.253],
       [ 0.466,  1.485,  0.873, -0.153],
       [-0.731, -0.978, -0.947, -0.808],
       [ 0.012,  0.261,  0.123,  0.086]], dtype=float32)

In [26]:
preprocessing_layer = tf.keras.layers.DenseFeatures(numeric_columns)

In [27]:
print(preprocessing_layer(example_batch).numpy()[0])

[-0.241  0.23   0.    -0.358]


In [28]:
CATEGORIES = {
    'username': ['BernieSanders', 'realDonaldTrump']
}


In [29]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [30]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='username', vocabulary_list=('BernieSanders', 'realDonaldTrump'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [31]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[0. 1.]


In [32]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)
preprocessing_layer = tf.keras.layers.DenseFeatures(numeric_columns)

In [33]:
print(preprocessing_layer(example_batch).numpy()[0])

[-0.241  0.23   0.    -0.358]


In [34]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

In [35]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [36]:
model.fit(train_data, epochs=20)

Epoch 1/20
1322/1322 [==============================] - 23s 17ms/step - loss: 0.3509 - accuracy: 0.0000e+00
Epoch 2/20
1322/1322 [==============================] - 10s 8ms/step - loss: 0.3448 - accuracy: 0.0000e+00
Epoch 3/20
1322/1322 [==============================] - 11s 8ms/step - loss: 0.3446 - accuracy: 0.0000e+00
Epoch 4/20
1322/1322 [==============================] - 9s 7ms/step - loss: 0.3443 - accuracy: 0.0000e+00
Epoch 5/20
1322/1322 [==============================] - 8s 6ms/step - loss: 0.3438 - accuracy: 0.0000e+00
Epoch 6/20
1322/1322 [==============================] - 11s 8ms/step - loss: 0.3440 - accuracy: 0.0000e+00
Epoch 7/20
1322/1322 [==============================] - 8s 6ms/step - loss: 0.3415 - accuracy: 0.0000e+00
Epoch 8/20
1322/1322 [==============================] - 12s 9ms/step - loss: 0.3412 - accuracy: 0.0000e+00
Epoch 9/20
1322/1322 [==============================] - 12s 9ms/step - loss: 0.3404 - accuracy: 0.0000e+00
Epoch 10/20
1322/1322 [================

In [37]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

110/110 [==============================] - 5s 44ms/step - loss: 0.2959 - accuracy: 0.0000e+00


Test Loss 0.2959033621983095, Test Accuracy 0.0


In [38]:
predictions = model.predict(test_data)

In [39]:

# Show some results
for prediction, ratio in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted sentiment: {:.02%}".format(prediction[0]),
        " | Actual outcome: "
            ('positive' if ratio > .01 else 'negative'))

TypeError: 'str' object is not callable

In [40]:
def convert_to_trigrams(tweets): 
    tokenizer = text.WhitespaceTokenizer()
    tokens = tokenizer.tokenize(tweets)
    # Ngrams, in this case tri-gram (n = 3)
    trigrams = text.ngrams(tokens, 3, reduction_type=text.Reduction.STRING_JOIN)
    print(trigrams.to_list())

In [41]:
# import csv

# with open('/Users/rachel/.keras/datasets/tweets3.csv', 'r') as f1:
#     csvlines = csv.reader(f1, delimiter=',')
#     for lineNum, line in enumerate(csvlines):
#         if not len(line)==6:
#             print(f"line:{lineNum} \t length:{len(line)}") 